# Proyecto de aprendizaje automático de extremo a extremo

### Trabajar con datos reales

Si queremos obtener resultados confiables, es fundamental trabajar con datos reales y no con datos inventados. Los modelos de aprendizaje automático solo pueden aprender patrones útiles si la información que reciben representa el mundo tal como es, con sus imperfecciones, ruido y complejidad.

Afortunadamente, existen múltiples plataformas que recopilan y publican conjuntos de datos abiertos. Muchas de ellas cuentan con comunidades activas que actualizan y comparten información de forma continua, lo que permite acceder a datos variados y de buena calidad.

Algunos de los repositorios de datos abiertos más populares son:

* **OpenML.org**
* **Kaggle.com**
* **PapersWithCode.com**

En este capítulo trabajaremos con el conjunto de datos **California Housing Prices**, que contiene información de un censo realizado en 1990. Nuestro objetivo será utilizar estos datos para construir un modelo capaz de predecir el precio de las viviendas en distintas zonas de California.

### Pipelines

Una **pipeline** es una secuencia de etapas automatizadas que transforman datos de forma organizada y reproducible.

En sistemas de *machine learning*, las pipelines son muy comunes cuando se necesita transformar, limpiar o manipular grandes volúmenes de datos antes de entrenar un modelo. En lugar de hacer cada paso manualmente, se encadenan procesos que se ejecutan uno tras otro.

Cada componente de la pipeline suele cumplir una función específica:

* Extraer datos (por ejemplo, desde una base de datos o un archivo).
* Procesarlos (limpieza, normalización, transformación, generación de características).
* Guardar el resultado en otro almacenamiento para que la siguiente etapa lo utilice.

En muchos sistemas modernos, estas etapas pueden ejecutarse de forma asíncrona o distribuida, lo que permite manejar grandes cantidades de información de manera eficiente.


### Antes de comenzar

Antes de iniciar un proyecto real, debemos preguntarnos cómo vamos a abordar el problema. No se trata solo de “entrenar algo”, sino de elegir correctamente el enfoque desde el principio.

En el caso de **California Housing Prices**, podemos plantearnos varias preguntas clave:

* ¿Qué tipo de aprendizaje necesitamos?
  ¿Supervisado, no supervisado, semi-supervisado, self-supervised o reinforcement learning?

* ¿Usaremos *batch learning* o *online learning*?

Empecemos por lo más sencillo:
¿Nuestro sistema recibirá un flujo constante de datos nuevos para actualizar el modelo? No. El conjunto de datos es histórico y está cerrado; queremos entrenar el modelo para hacer predicciones sobre ese mismo tipo de información. No necesitamos que el modelo se actualice continuamente con datos nuevos, así que utilizaremos **batch learning**.

Ahora, ¿qué tipo de aprendizaje es?
Tenemos el precio de las casas (la variable objetivo) y múltiples características asociadas a cada vivienda. Es decir, conocemos la respuesta correcta durante el entrenamiento. Por lo tanto, se trata claramente de **aprendizaje supervisado**.

Finalmente, ¿qué tipo de modelo utilizaremos?
El comportamiento que buscamos modelar es esencialmente lineal. Sin embargo, no tenemos una sola variable explicativa, sino varias características que influyen en el precio. Por esta razón, no basta con una regresión lineal simple: utilizaremos **regresión lineal múltiple**, que nos permite considerar múltiples variables de entrada al mismo tiempo.